# Decorators

A decorator takes a bare `Callable` as input and returns another decorated `Callable` as output.

In [1]:
from typing import Callable

In [2]:
def decorator(func: Callable) -> Callable:
    def decorated_func(*args, **kwargs):
        print('Custom stuff before the udf is called')
        retval = func(*args, **kwargs)  # closure on func
        print('Custom stuff after the udf has been called')
        return retval
    return decorated_func

In [3]:
@decorator
def bake(flavor: str, gluten_free=False):
    print(f'Baking {flavor} cake that is {gluten_free} gluten free')

In [4]:
bake('Chocolate')

Custom stuff before the udf is called
Baking Chocolate cake that is False gluten free
Custom stuff after the udf has been called


The above function definition is equivalent to - 
```python
bake = decorator(bake)
```
This is calling the `decorator` function with the bare function as the parameter, and then substituting the function value with the new decorated function returned by the `decorator`.

It is important for the decorated function to be defined inside the decorator because it needs to use passed in bare `func`, and we cannot include that in the decorated function's signature.

In [5]:
def freeze(flavor: str):
    print(f'Freezing {flavor} ice cream')

freeze = decorator(freeze)

In [6]:
freeze('Strawberry')

Custom stuff before the udf is called
Freezing Strawberry ice cream
Custom stuff after the udf has been called


A parameterized decorator is nothing but a decorator factory, i.e., it takes some parameters to create a decorator and then returns that decorator.

In [7]:
Decorator = Callable[[Callable], Callable]

def decorator_factory(temperature: int) -> Decorator:
    def decorator(func: Callable) -> Callable:
        def decorated_func(*args, **kwargs):
            print(f'Preheating oven to {temperature} degress')  # Closure on temperature
            retval = func(*args, **kwargs)  # Closure on func
            print('Switching off oven')
            return retval
        return decorated_func
    return decorator

In [8]:
@decorator_factory(440)
def bake(flavor: str, gluten_free=False):
    print(f'Baking {flavor} cake that is {gluten_free} gluten free')

In [9]:
bake('Chocolate')

Preheating oven to 440 degress
Baking Chocolate cake that is False gluten free
Switching off oven


This is equivalent to calling -
```python
bake = decorator_factory(440)(bake)
```
As can be seen `decorator_factory` is not really a decorator, instead it creates and returns a decorator. And remember a decorator is nothing but a function that takes in a bare function and returns another decoratorated function (or strictly speaking it is a function that takes in a Callable and returns a Callable).

## Class Decorators
Decorators can be used to decorate classes as well.

In [10]:
def class_decorator(klass):
    def decorated_class(*args, **kwargs):
        print('Custom stuff before the class initializer is called')
        obj = klass(*args, **kwargs)
        print('Custom stuff after the class initializer has been called')
        return obj
    return decorated_class

In [11]:
@class_decorator
class Cookie:
    def __init__(self, flavor):
        print('Initializing Cookie')
        self.flavor = flavor
    
    def __repr__(self):
        return f'<Cookie(flavor={self.flavor})>'

In [12]:
cc = Cookie('Chocolate Chip')
cc

Custom stuff before the class initializer is called
Initializing Cookie
Custom stuff after the class initializer has been called


<Cookie(flavor=Chocolate Chip)>

As with the function decorator, the class declaration is equivalent to -
```python
Cookie = class_decorator(Cookie)
```

In [13]:
class Cake:
    def __init__(self, flavor):
        print('Initializing Cake')
        self.flavor = flavor
        
    def __repr__(self):
        return f'<Cake(flavor={self.flavor})>'
    

Cake = class_decorator(Cake)

In [14]:
choc = Cake('Chocolate')
choc

Custom stuff before the class initializer is called
Initializing Cake
Custom stuff after the class initializer has been called


<Cake(flavor=Chocolate)>

As can be seen from the code of `class_decorator` and `decorator`, they are virtually identical. In fact, the same decorator can be used for both functions and classes.

In [15]:
@decorator
class Doughnut:
    def __init__(self, flavor):
        print('Initializing Doughnut')
        self.flavor = flavor
    
    def __repr__(self):
        return f'<Doughnut(flavor={self.flavor})>'

In [16]:
maple = Doughnut('Maple')
maple

Custom stuff before the udf is called
Initializing Doughnut
Custom stuff after the udf has been called


<Doughnut(flavor=Maple)>

This is possible because a decorator is nothing but a function that takes in a bare Callable and returns a decorated Callable. Whether that Callable is a function or a class does not matter.